## Mounting of Google Drive for use in  Google Colab

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## IMPORTATION OF REQUIRED LIBRARIES

In [3]:
!pip install torch -q
import pandas as pd 
import plotly.express as px
import random
import matplotlib.pyplot as plt
import numpy as np
import torch
import time
from torch import Tensor,deg2rad,sin,asin,cos,sqrt,min,max#
from torch.utils.data import DataLoader 

## LOADING MY DATA

In [4]:
pos_data=pd.read_csv('/content/drive/MyDrive/BtEurope.csv') #reading from the raw Csv
pos_data.drop_duplicates(inplace=True)
pos_tensors=Tensor(pos_data[['Lat','Lon']].values) #converting the longitudes and the latitudes to tensor format

In [5]:
pos_data

,Lon,Lat,City,Country
0,19.03991,47.49801,Budapest,Hungary
1,11.57549,48.13743,Munich,Germany
2,14.42076,50.08804,Prague,Czech Republic
3,16.37208,48.20849,Vienna,Austria
4,6.77616,51.22172,Dusseldorf,Germany
5,8.68333,50.11667,Frankfurt,Germany
6,8.55000,47.36667,Zurich,Switzerland
7,2.34880,48.85341,Paris,France
8,9.18951,45.46427,Milan,Italy
9,2.15899,41.38879,Barcelona,Spain


## PLOTTING THE LOCATIONS OF THE MAP OF EUROPE

In [6]:
fig = px.scatter_geo(pos_data,lat='Lat',lon='Lon', hover_name="City",scope='europe')
fig.update_layout(title = 'Eurpean Cities', title_x=0.5)
fig.show()

## DEFINING MY  DISTANCE FUNCTION

In [7]:
# THIS FUNCTION CALCULATES THE HAVERSINE DISTANCE BETWEEN TWO POINTS
def haversine(loc1, loc2):
      R = 3959.87433 # this is in miles.  For Earth radius in kilometers use 6372.8 km
      lat1=loc1[:,0]
      lon1=loc1[:,1]
      lat2=loc2[0]
      lon2=loc2[1]
      dLat = deg2rad(lat2 - lat1)# CONVERTING DEGREES TO RADIANS
      dLon = deg2rad(lon2 - lon1)
      lat1 = deg2rad(lat1)
      lat2 = deg2rad(lat2)
      a = sin(dLat/2)**2 + cos(lat1)*cos(lat2)*sin(dLon/2)**2
      c = 2*asin(sqrt(a))
      return R * c # RETURNS THE HAVERSINE DISTANCE

## DEFINING OUR OBJECTIVE FUNCTIONS

In [8]:
k,l=1,1
m,n=0.01,0.01
p=0.001
q=0.3

In [9]:
# THE FIRST OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS THE MAXIMUM OF THE MINIMUM NODE TO CONTROLLER LATENCY
def maxn2C_fn(city_locs, cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  min_dists = dists
  for i in range(1,7):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = min(min_dists, dists)
  return min_dists.max()
# THE SECOND OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS THE MEAN OF THE MINIMUM NODE TO CONTROLLER LATENCY
def mean2C_fn(city_locs, cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  min_dists = dists
  for i in range(1,7):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = min(min_dists, dists)
    return min_dists.mean()
# THE THIRD OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS MAXIMUM INTERCONTROLLER LATENCY
def maxC2C_fn(cont_locs,amb_num):
  dists = haversine(cont_locs,amb_num[0])
  max_dists = dists
  for i in range(1,7):
    dists = haversine(cont_locs,amb_num[i])
    max_dists = max(max_dists, dists)
  return max_dists.max()*0.01
# THE FOURTH OBJECTIVE FUNCTION HERE CALCULATES AND RETURNS DIVISION OF THE SUM OF INTERCONTROLLER LATENCY BY 5C2
def meanC2C_fn(cont_locs,amb_num):
  dists = haversine(cont_locs,amb_num[0])
  sum_dists = dists.sum()
  for i in range(1,7):
    dists = haversine(cont_locs,amb_num[i]).sum()
    sum_dists = sum_dists+ dists
  return sum_dists/20*0.01
def load_balancing(city_locs,cont_locs):
   node_counts=np.zeros(7)
   for i in range(city_locs.shape[0]):
    dists = haversine(cont_locs,city_locs[i])
    node_counts[np.argmin(dists.clone().detach().numpy())]+=1
   return np.ptp(node_counts)
def max_tolerance(city_locs,cont_locs):
  dists = haversine(city_locs,cont_locs[0])
  max_dists = dists
  for i in range(1,7):
    dists = haversine(city_locs,cont_locs[i])
    min_dists = torch.max(max_dists, dists)
  return max_dists.max()
def all_losses(city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs).item()
  l2=mean2C_fn(city_locs,cont_locs).item()
  l3=maxC2C_fn(cont_locs,cont_locs).item()
  l4=meanC2C_fn(cont_locs,cont_locs).item()
  l5=max_tolerance(city_locs,cont_locs).item()
  l6=load_balancing(city_locs,cont_locs)
  return np.array([l1,l2,l3,l4,l5,l6])
  #return np.array([l1,l2,l3,l4])

In [10]:
def overall_loss(city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs)
  l2=mean2C_fn(city_locs,cont_locs)
  l3=maxC2C_fn(cont_locs,cont_locs+0.0001)
  l4=meanC2C_fn(cont_locs,cont_locs+0.0001)
  l5=max_tolerance(city_locs,cont_locs)
  return k*l1 + l*l2+m*l3+n*l4
def overall_metric( city_locs,cont_locs):
  l1=maxn2C_fn(city_locs,cont_locs).item()
  l2=mean2C_fn(city_locs,cont_locs).item()
  l3=maxC2C_fn(cont_locs,cont_locs+0.001).item()
  l4=meanC2C_fn(cont_locs,cont_locs+0.001).item()
  l5=max_tolerance(city_locs,cont_locs).item()
  l6=load_balancing(city_locs,cont_locs)
  return (k*l1 + l*l2+m*l3+n*l4+p*l5)*(l6**q)

## Specify your preferred starting seed and population size

In [11]:
start=419 # starting seed No
size=2500 # pop_size

## GENERATING OUR RANDOM INITIAL LOCATIONS 

In [12]:
df=pd.DataFrame(index=range(size))

In [13]:
from numpy.ma.core import copy
def loc_to_cities(cont_locs):
  cont_locs=cont_locs.detach()
  min_dists=[]
  copy=pos_tensors.clone()
  for loc in cont_locs:
    dists=haversine(copy,loc).detach().numpy()
    row_exclude = np.argmin(dists)
    copy[row_exclude]=Tensor([-999,-999])
    min_dists.append(np.argmin(dists))
  return min_dists

In [14]:
import random,numpy
from tqdm import tqdm
from torch import manual_seed
# seeding the environment to enable reproductibility of results
for siz in range(size):
  seed=siz+start
  np.random.seed(seed)
  rand_cities =np.random.choice(range(pos_data.shape[0]),7,replace=False).astype(int)
  for i in range(7):
      df.loc[siz,'input_' +str(i)]=rand_cities[i]

## TRAINING THE MODEL 

In [15]:
# setting our training parameters 
lr=0.01
n_epochs = 100
main_seed=12
# dataloader to load our data in batches, here we are using a batch size of 1
for siz in tqdm(range(size)):
  seed=siz+start
  manual_seed(12)
  # using torch to generate a random of size 7 by 2
  rand_cities =df[df.columns[:7]].values[siz]
  rand_cities=np.sort(rand_cities)
  cont_locs=Tensor(pos_data[['Lat','Lon']].values[rand_cities.astype(int)])# adding the mean of our longitudes and latitudes to enable faster convergence of the model 
  initial_locs = cont_locs.clone() # Save originals for later
  cont_locs=cont_locs+0.000001 # adding little noise for calculation of gradients
  cont_locs.requires_grad_(True)
  dl = DataLoader(pos_tensors, batch_size=10, shuffle=False)
# Store loss over time
  #train_losses = []
  val_losses = []
  losses_sum=[]
  verbose=4
  early_stopping=10
  #best_loss=initial_losses
# for keeping the best locs 
  final_locs=cont_locs.clone()
  for epoch in range(n_epochs):
   # Run through batches
    for pos_tensor in dl:
      loss2 = (meanC2C_fn(cont_locs,cont_locs+0.00001)+maxC2C_fn( cont_locs,cont_locs+0.00001))*0.1# calculating the intercontroller objective functions
      loss =loss2+ (mean2C_fn(pos_tensors, cont_locs)+maxn2C_fn(pos_tensor, cont_locs))# calculating the intercontroller objective functions
      loss.backward() # Calc grads
      cont_locs.data -= lr * cont_locs.grad.data # Update locs
      cont_locs.grad = None # Reset gradients for next step
    #train_losses.append(all_losses(pos_tensor,cont_locs))
    val_loss=all_losses(pos_tensors,cont_locs)
    losses=val_loss.sum()
    if epoch>0 :
      if losses< np.min(losses_sum):
         final_locs=cont_locs.clone()
         best_loss=val_loss
    val_losses.append(val_loss)
    losses_sum.append(losses)
    #if epoch % verbose==0:
     # print("After {0:.0f} epochs: maxN2c: {1:.2f} meanN2C: {2:.2f} maxICL: {3:.2f} meanICL: {4:.2f} Overall: {5:.2f} ".format(epoch,val_loss[0],val_loss[1],val_loss[2],val_loss[3],losses))
    if epoch > early_stopping:
      if np.min(losses_sum[-early_stopping:])> np.min(losses_sum):
      #  print ('Training ended by early stopping, best score at epoch {}'.format(epoch))
         break
  for i in range(7):
    df.loc[siz,'output_'+str(i)]=loc_to_cities(final_locs)[i]
  #print("Best scores at epoch {0} maxN2c: {1:.2f} meanN2C: {2:.2f} maxICL: {3:.2f} meanICL: {4:.2f} Overall: {5:.2f} ".format(epoch,best_loss[0],best_loss[1],best_loss[2],best_loss[3],best_loss.sum()))

100%|██████████| 2500/2500 [40:26<00:00,  1.03it/s]


In [16]:
df.to_csv('rcgautograd23.csv',index=False)

In [17]:
df

,input_0,input_1,input_2,input_3,input_4,input_5,input_6,output_0,output_1,output_2,output_3,output_4,output_5,output_6
0,10.0,12.0,0.0,17.0,11.0,19.0,2.0,2.0,6.0,16.0,11.0,20.0,17.0,19.0
1,2.0,1.0,16.0,13.0,9.0,0.0,12.0,0.0,5.0,2.0,9.0,12.0,13.0,16.0
2,10.0,18.0,14.0,13.0,5.0,3.0,9.0,2.0,6.0,9.0,10.0,20.0,14.0,18.0
3,18.0,11.0,0.0,16.0,20.0,3.0,8.0,0.0,5.0,8.0,9.0,16.0,17.0,20.0
4,20.0,16.0,3.0,11.0,10.0,4.0,2.0,18.0,1.0,4.0,10.0,11.0,16.0,20.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2495,14.0,12.0,3.0,20.0,5.0,6.0,7.0,3.0,17.0,6.0,9.0,12.0,14.0,20.0
2496,6.0,3.0,19.0,14.0,7.0,2.0,0.0,3.0,17.0,0.0,6.0,9.0,14.0,20.0
2497,2.0,13.0,20.0,3.0,19.0,1.0,9.0,14.0,2.0,1.0,9.0,13.0,19.0,20.0
2498,4.0,16.0,5.0,8.0,20.0,14.0,17.0,18.0,1.0,9.0,14.0,16.0,17.0,20.0
